In [1]:
! pip install segmentation-models-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 5.1 MB/s 
     |████████████████████████████████| 376 kB 19.3 MB/s 
     |████████████████████████████████| 58 kB 6.9 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=d9f3acbcb18a0ac2d3ed7424be1b450315dac8c7d4378b90d79d4cba026a9ca0
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=fe593400c4079701b3860f5667514646b9c13a3dadd769864f99d3744f859c65
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels


In [2]:
! pip install git+https://github.com/qubvel/segmentation_models.pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/qubvel/segmentation_models.pytorch to /tmp/pip-req-build-tkv26a3j
  Running command git clone -q https://github.com/qubvel/segmentation_models.pytorch /tmp/pip-req-build-tkv26a3j
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for segmentation-models-pytorch: filename=segmentation_models_pytorch-0.3.0.dev0-py3-none-any.whl size=98008 sha256=73693e8b65e0c3c5509c7302257b892ce967205ae8c66c4c100307b51125f58c
  Stored in directory: /tmp/pip-ephem-wheel-cache-aihaf7l2/wheels/fa/c5/a8/1e8af6cb04a0974db8a4a156ebd2fdd1d99ad2558d3fce49d4
Successfully built segmentation-models-pytorch
  Attempting uninstall: segmentation-models-pytorch
    Found existing installation: segmentation-models-pytorch 0.2.1
    Uninstalling segmentation-models-pytorch-0.2.1:
      Successfully uninsta

In [3]:
#Importations for datasets
import torch
from torch.utils.data import Dataset
from torchvision import datasets, models
from torchvision import transforms as T
from torchvision.transforms import ToTensor
import torchvision
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import PIL
from PIL import Image
from glob import glob
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import torch.nn.functional as F

In [4]:
#GET Data
from google.colab import files
files.upload() #Upload kaggle.json here.

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [5]:
#set up directories and unzip
! mkdir aisegment-1
% cd aisegment-1/
!kaggle datasets download -d vincenttu/aisegment-split-1
!unzip -q aisegment-split-1.zip
!rm aisegment-split-1.zip
% cd ../

/content/aisegment-1
 99% 3.54G/3.56G [00:31<00:00, 179MB/s]
100% 3.56G/3.56G [00:31<00:00, 121MB/s]
/content


In [6]:
#Unzip 
!mkdir aisegment-split-1
%cd aisegment-split-1
!kaggle datasets download -d vincenttu/aisegment-split-1
!unzip -q aisegment-split-1.zip
!rm aisegment-split-1.zip
%cd

/content/aisegment-split-1
 99% 3.54G/3.56G [00:19<00:00, 223MB/s]
100% 3.56G/3.56G [00:19<00:00, 196MB/s]
/root


In [7]:
#set up paths
filepaths = list(glob('/content/aisegment-split-1/*'))

#Extract data and label sets
label_paths = []
image_paths = []
for path in filepaths:
  if 'label' in path:
    label_paths.append(path)
  else:
    image_paths.append(path)
  
for x in label_paths:
  y = x.split("_")[1]
  y = '/content/aisegment-split-1/' + y
  if y not in image_paths:
    label_paths.remove(x)

#remove bad data
label_paths.remove('/content/aisegment-split-1/label_1803201916-00000002.jpg')
image_paths.remove('/content/aisegment-split-1/1803201916-00000002.jpg')

#sort data
label_paths.sort()
image_paths.sort()

In [8]:
from torchvision.transforms.transforms import ToTensor
class SegmentDataset(Dataset):
  def __init__(self, images_path, labels_path):
    self.images_path = images_path
    self.labels_path = labels_path

    self.transform_both = T.Compose([
        T.ToTensor(),            
        T.Resize((128, 128)) #want to make square, messes up ration but all good     
    ])

    self.transform_image = T.Compose([
        T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

    self.transform_label = T.Compose([
                            
    ])

  def __len__(self):
    return len(self.images_path)

  def __getitem__(self, idx):
    image = PIL.Image.open(self.images_path[idx])
    label = PIL.Image.open(self.labels_path[idx])

    image_array = np.asarray(image)

    #make into array, get only alpha color channel, expand to make 3D(with C.C.)
    label_array = np.expand_dims(np.asarray(label)[:,:,3], axis=2)
    
    image_tensor = self.transform_both(image_array)
    label_tensor = self.transform_both(label_array)

    #only transform images (training set) cause it goes through model
    image_tensor = self.transform_image(image_tensor)
    label_tensor = self.transform_label(label_tensor)

    label_tensor = (label_tensor>0).type(torch.FloatTensor)  # np.float32, float

    return image_tensor, label_tensor

In [9]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [22]:
#Variable instatiation
batch_size = 32
data = SegmentDataset(image_paths, label_paths)

train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])

loaders = {
    'train' : torch.utils.data.DataLoader(train_dataset, 
                                          batch_size=32, 
                                          shuffle=True, 
                                          num_workers=1),
    
    'test'  : torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=32, 
                                          shuffle=False, 
                                          num_workers=1),
}
#loaders

device = 'cuda' if torch.cuda.is_available() else 'cpu'

cool_model = model.train().to(device)

num_epochs = 5
learning_rate = 0.01

optimizer = torch.optim.NAdam(model.parameters(), lr=learning_rate)
loss_func=nn.BCELoss()

In [25]:
#Training Loop
def train_model(model, criterion, optimizer, num_epochs=3):
  for epoch in range(num_epochs):
      for i, batch in tqdm(enumerate(loaders['train'])):
          image, label = batch 
          image = image.to(device)
          label = label.to(device)

          # Forward pass
          outputs = model(image)

          loss = F.binary_cross_entropy(torch.sigmoid(outputs), label)
          
          optimizer.zero_grad()
          # Backward and optimize
          loss.backward()
          optimizer.step()


      print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
trained_model=train_model(model,nn.BCELoss,optimizer)

0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:133: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


KeyboardInterrupt: ignored

In [26]:
def test():
    # Test the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in loaders['test']:
            test_output, last_layer = model(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))
            pass
        print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)
    
pass
test()

TypeError: ignored

In [24]:
from torch.autograd import Variable
#num_epochs = 10
def train(num_epochs, gcnn, loaders):
    
    model.train()
        
    # Train the model
    total_step = len(loaders['train'])
        
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            
            # gives batch data, normalize x when iterate train_loader
            b_x = Variable(images)   # batch x
            b_y = Variable(labels)   # batch y
            output = model(b_x)[0]               
            loss = loss_func(output, b_y)
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            
            # backpropagation, compute gradients 
            loss.backward()    
            # apply gradients             
            optimizer.step()                
            
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
            pass
        
        pass
    
    
    pass
train(num_epochs, model, loaders)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f64babb1320>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:133: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest 

ValueError: ignored

In [8]:
x = torch.rand(1, 3, 512, 256)

model.eval()
with torch.no_grad():
    z = model(x)

print(z.shape)

torch.Size([1, 1, 512, 256])
